## Import libraties

In [1]:
import os 
import csv
import numpy as np
import pandas as pd
import random # library to generate random numbers
np.random.seed(seed=42)
import math

#### Import data

In [2]:
data1 = pd.read_csv("cities_and_distances.csv",header=0,index_col=0)
data1

,Bangalore,Jaipur,Varanasi,Udaipur,Delhi,Chennai,Mysore,Agra,Kolkata,Mumbai,Hyderabad
Bangalore,0,2026,1825,1724,2166,346,149,1923,1862,980,569
Jaipur,2026,0,849,393,287,2108,2121,240,1520,1146,1483
Varanasi,1825,849,0,1154,834,1862,1976,614,683,1524,1237
Udaipur,1724,393,1154,0,661,2077,1809,634,1825,753,1349
Delhi,2166,287,834,661,0,2203,2317,233,1498,1415,1578
Chennai,346,2108,1862,2077,2203,0,482,1957,1669,1345,627
Mysore,149,2121,1976,1809,2317,482,0,2072,2012,1064,718
Agra,1923,240,614,634,233,1957,2072,0,1280,1323,1334
Kolkata,1862,1520,683,1825,1498,1669,2012,1280,0,2053,1493
Mumbai,980,1146,1524,753,1415,1345,1064,1323,2053,0,711


#### Starting Point is Bangalore

In [3]:
starting_point = "Bangalore"

##### Format the data into required format

In [4]:
# data1.reset_index(inplace=True)
# data1  = data1.iloc[:,2:]
# data1.index = data1.columns.values


###### The number of possible solutions are: 10! = 3628800
We can generate all possible solutions and try to find the best of those , but this is a time taking process and may not be feasible for big problems.
Genetic algorithm approach takes a randominzed search approach to find the best solution, with enough number of iterations, mutation and cross over, GA can reach the global minima for most of the problems.

##### Map places to integers

In [5]:
cities_mapping = {}

for i in enumerate(data1.columns):
    cities_mapping[i[0]] = i[1]
cities_mapping


{0: 'Bangalore',
 1: 'Jaipur',
 2: 'Varanasi',
 3: 'Udaipur',
 4: 'Delhi',
 5: 'Chennai',
 6: 'Mysore',
 7: 'Agra',
 8: 'Kolkata',
 9: 'Mumbai',
 10: 'Hyderabad'}

In [6]:
one_route = [5,6,10,9,1,3,4,7,2,8]# one of the routes

In [7]:
one_route

[5, 6, 10, 9, 1, 3, 4, 7, 2, 8]

## createRandomRoute:: this function to generate random route

In [8]:
def createRandomRoute():
    """ This function generates a route by sampling numbers """
    my_randoms = random.sample(range(1,11), 10)    # excluding 0 from the range as 0 is the starting point for us i.e Bangalore
    return my_randoms

In [9]:
temp_route = createRandomRoute()

In [10]:
temp_route

[2, 6, 5, 8, 4, 1, 10, 3, 9, 7]

## fitnessFunction :: Function to compute the route cost using the already defined path 

In [11]:
def fitnessFunction(pvRouteMap):
    """This functions takes the route generated and returns the cost incurred."""
    traverseData    = data1.copy() # creating a copy of the original df
    sourcePoint     = 0 # defining a starting point
    stopsCovered    = 0 # setting the number of stops covered to 0
    routeCost       = 0 # setting the initial cost to 0
    pvRouteMap1 = pvRouteMap #+ [pvRouteMap[0]]
#     print(traverseData)
    # initiate a while loop and calculate the cost for the whole path traversed and return the cost
    while(stopsCovered < len(pvRouteMap1)) :  
        routeCost = routeCost + traverseData.iloc[sourcePoint, pvRouteMap1[stopsCovered]]   
        """route cost is the sum of the cost incurred from travelling from one point to the next according
        to the route that was generated previously."""
        sourcePoint = pvRouteMap1[stopsCovered] # update the source point to the next point in the route
        stopsCovered = stopsCovered+1 #Adding 1 to the stops covered
#         print(sourcePoint)
    routeCost = routeCost + traverseData.iloc[pvRouteMap1[-1],0]

    return(routeCost)

In [12]:
fitnessFunction(temp_route)

14568

#### We can look at the positions and think of one good route , we will see if our algorithm performs better than this route

In [13]:
one_route_cities = []
for i in one_route:
    one_route_cities.append(cities_mapping[i])
one_route_cities

['Chennai',
 'Mysore',
 'Hyderabad',
 'Mumbai',
 'Jaipur',
 'Udaipur',
 'Delhi',
 'Agra',
 'Varanasi',
 'Kolkata']

##  mutateRoute:: this function will generate mutated path where we define mutation policy.

In [14]:
def mutateRoute(initPath, mutateFactor):
    """ This functions generates a mutated path , takes an input path and returns a mutated path.
    Mutate factor is the point at which the string(route here) is split and the two parts are swapped."""
#    try:
    temp1 = initPath[0:mutateFactor]
    temp2 = initPath[mutateFactor:len(initPath)]
    newPath = temp2 + temp1
#     except:
#         temp1 = initPath[0:max(mutateFactor)]
#         temp2 = initPath[max(mutateFactor):]
#         newPath = temp2 + temp1
    return(newPath)

#### Generate a sample path and mutate that path 

In [15]:
temp_route = createRandomRoute()

In [16]:
temp_route

[2, 6, 10, 9, 5, 1, 4, 7, 8, 3]

In [17]:
mutateRoute(temp_route,2)

[10, 9, 5, 1, 4, 7, 8, 3, 2, 6]

In [18]:
fitnessFunction(temp_route)

14032

In [19]:
print("Total distance travelled to cover the route of {} is {}".format(" => ".join( one_route_cities ),fitnessFunction(one_route)))

Total distance travelled to cover the route of Chennai => Mysore => Hyderabad => Mumbai => Jaipur => Udaipur => Delhi => Agra => Varanasi => Kolkata is 7849


# Function to create Initial (random) Population 

In [20]:
def initialPopCost(ninitpop = 10):
    """This function generates new routes and calculates the cost of the route.
    returns a dictionary of routes and the cost."""
    np.random.seed(seed=42)
    intial_cost = 0
    routeCost = []
    routes = [createRandomRoute() for i in range(0, ninitpop)]
    routes_Cost = [fitnessFunction(i) for i in routes]
    routes_DF = pd.DataFrame([routes,routes_Cost]).transpose()
    routes_DF.columns = ['Route','Cost']
    return(routes_DF)
# initialPopCost()

In [21]:
initial_pop_cost = initialPopCost(10)
sorted_init_pop = initial_pop_cost.sort_values(['Cost'])

In [22]:
initial_pop_cost

,Route,Cost
0,"[10, 3, 2, 8, 1, 7, 4, 9, 6, 5]",9055
1,"[7, 1, 4, 3, 9, 5, 2, 10, 8, 6]",11962
2,"[5, 2, 7, 8, 4, 10, 6, 9, 3, 1]",12132
3,"[5, 1, 9, 8, 4, 10, 6, 7, 2, 3]",15011
4,"[4, 3, 2, 7, 1, 9, 10, 8, 5, 6]",10485
5,"[9, 6, 2, 5, 3, 7, 8, 10, 4, 1]",15257
6,"[3, 9, 8, 1, 5, 7, 4, 2, 10, 6]",13286
7,"[8, 9, 4, 1, 10, 5, 7, 3, 6, 2]",15928
8,"[10, 3, 9, 2, 5, 6, 7, 4, 8, 1]",13888
9,"[1, 3, 2, 9, 5, 10, 6, 4, 7, 8]",13479


In [23]:
sorted_init_pop.head()

,Route,Cost
0,"[10, 3, 2, 8, 1, 7, 4, 9, 6, 5]",9055
4,"[4, 3, 2, 7, 1, 9, 10, 8, 5, 6]",10485
1,"[7, 1, 4, 3, 9, 5, 2, 10, 8, 6]",11962
2,"[5, 2, 7, 8, 4, 10, 6, 9, 3, 1]",12132
6,"[3, 9, 8, 1, 5, 7, 4, 2, 10, 6]",13286


## Partially Mapped Crossover

###### - implemented PMX conceptualized by goldberg

In [24]:
def crossOverFunction(parent1, parent2,crossover_factor_start_pos=2,
                             crossover_factor_end_pos=6):
    indexes_for_crossover = random.sample(set(range(len(p1))), 2)
    crossover_factor_start_pos,crossover_factor_end_pos = min(indexes_for_crossover),max(indexes_for_crossover)
#     print (indexes_for_crossover)
    ## generate child 1
    child1 = parent1[0:crossover_factor_start_pos]+\
    parent2[crossover_factor_start_pos:crossover_factor_end_pos] +\
    parent1[crossover_factor_end_pos:]
    
    ## generate child 2
    child2 = parent2[0:crossover_factor_start_pos] +\
    parent1[crossover_factor_start_pos:crossover_factor_end_pos] +\
    parent2[crossover_factor_end_pos:]
    
    ## Create mappings
    mpping = list(zip(parent1[crossover_factor_start_pos:crossover_factor_end_pos], # create tuple of mappings
                      parent2[crossover_factor_start_pos:crossover_factor_end_pos]))
#     print(mpping)
    # run until all the nodes in the route are unique
    while len(np.unique(child1)) != len(child1):
        child1_part = child1[:crossover_factor_start_pos]+child1[crossover_factor_end_pos:]
        for i in child1_part:
            for j in mpping:
                if i == j[1]: #j[1] - Parent 2 window elements                 
                    child1_part[child1_part.index(i)] = j[0] #j[0] - Parent 1 window elements
                    
        child1 = child1_part[:crossover_factor_start_pos]+ child1[crossover_factor_start_pos:crossover_factor_end_pos]+ child1_part[crossover_factor_start_pos:]
        
#         print("Child1 Intermediate {}".format(child1))
#     print("Child1 final {}".format(child1))

#     print("Child2 original {}".format(child2))

# run until all the nodes in the route are unique
    while len(np.unique(child2)) != len(child2):
        child2_part = child2[:crossover_factor_start_pos]+child2[crossover_factor_end_pos:]
        for i in child2_part:
            for j in mpping:
                if i == j[0]:
                    child2_part[child2_part.index(i)] = j[1]
        child2 = child2_part[:crossover_factor_start_pos]+         child2[crossover_factor_start_pos:crossover_factor_end_pos]+         child2_part[crossover_factor_start_pos:]
#         print("Child2 Intermediate {}".format(child2))
#     print("Child2 final {}".format(child2))
    return( child1,child2)

In [25]:
p1 = [5,7,2,4,8,9,3,6]

In [26]:
p2 = [8,6,9,3,2,5,7,4]

In [27]:
for i in zip([p1,p2],crossOverFunction(p1,p2)):
    print ("Parent: {}  : Child: {}".format(i[0],i[1]))

Parent: [5, 7, 2, 4, 8, 9, 3, 6]  : Child: [8, 4, 9, 3, 2, 5, 7, 6]
Parent: [8, 6, 9, 3, 2, 5, 7, 4]  : Child: [5, 6, 2, 4, 8, 9, 3, 7]


 ### Best Solution So Far


Total distance travelled to cover the final route of 
 Bangalore => Chennai => Hyderabad => Kolkata => Varanasi => Delhi => Agra => Jaipur => Udaipur => Mumbai => Mysore => Bangalore is 6815


In [28]:
def genetic_algorithm(sorted_current_generation, no_of_chromosomes = 100 ,mutation_factor=0.2, xover_factor = 0.3):

    
    nextgen_childs = [] 
    
    xover_pool = int(round(no_of_chromosomes * xover_factor ))
    no_of_loops = int((no_of_chromosomes // 2) )
    
    print ('starting crossover')
    for i in range(0,no_of_loops):
        parent_index1 = random.choice(range(0,xover_pool))
        p1= sorted_current_generation.iloc[parent_index1,0]
        parent_index2 = random.choice(range(0,xover_pool))
        p2= sorted_current_generation.iloc[parent_index2,0]

        c1, c2 = crossOverFunction(p1,p2)
        nextgen_childs.extend([c1,c2])
        
    nextgen_costs = [fitnessFunction(i) for i in nextgen_childs]
    next_generation = pd.DataFrame([nextgen_childs,nextgen_costs],).transpose() 
    next_generation.columns = ['Route','Cost']
    
    print('starting Mutation')
    
    for i in range(0, no_of_chromosomes) :
        mutation_prob = random.random()
        if mutation_prob < mutation_factor: 
            mutated_child = mutateRoute (next_generation.iloc[i,0] , 2)
            next_generation.iloc[i,0] = mutated_child
            next_generation.iloc[i,1] = fitnessFunction(next_generation.iloc[i,0])

    return (next_generation)

In [29]:
def perform_genetic_algorithm(no_of_iterations = 20, no_of_chromosomes = 100 ,mutation_factor=0.2, xover_factor = 0.3):
    best_solution = []
    best_cost = 99999
    current_generation = initialPopCost(no_of_chromosomes)
    sorted_current_generation = current_generation.sort_values('Cost')
    for i in range(0, no_of_iterations) :
        next_generation = genetic_algorithm(sorted_current_generation,no_of_chromosomes  ,mutation_factor, xover_factor)
        sorted_next_generation = next_generation.sort_values('Cost')
        print ("Completed {} runs of GA.".format(i + 1))
        print ("below are the best chromosomes:")
        print(sorted_next_generation.head(3))
#         if sorted_next_generation.iloc[0,1] > sorted_current_generation.iloc[0,1] :
#             sorted_current_generation = sorted_next_generation
#         else :
#             i = no_of_iterations +1
#             print ("Completed {} runs of GA.".format(i + 1))")
        sorted_current_generation = sorted_next_generation
        if best_cost > sorted_next_generation.iloc[0,1] :
            best_cost = sorted_next_generation.iloc[0,1]
            best_solution = sorted_next_generation.iloc[0,0]

        
        final_route = []
        for i in best_solution:
            final_route.append(cities_mapping[i])
        final_route = [starting_point] + final_route + [starting_point]
        print("Total distance travelled to cover the final route of \n {} is {}".format(" => ".join(final_route),best_cost))
            

    
    return (best_solution, best_cost)

In [30]:
best_solution, best_cost = perform_genetic_algorithm()

starting crossover
starting Mutation
Completed 1 runs of GA.
below are the best chromosomes:
                              Route   Cost
9   [10, 8, 7, 2, 4, 1, 9, 3, 6, 5]   9613
59  [10, 3, 9, 8, 2, 4, 7, 1, 6, 5]   9663
45  [9, 7, 1, 4, 2, 8, 5, 6, 10, 3]  10289
Total distance travelled to cover the final route of 
 Bangalore => Hyderabad => Kolkata => Agra => Varanasi => Delhi => Jaipur => Mumbai => Udaipur => Mysore => Chennai => Bangalore is 9613
starting crossover
starting Mutation
Completed 2 runs of GA.
below are the best chromosomes:
                              Route   Cost
91  [7, 1, 3, 4, 8, 2, 10, 5, 9, 6]   9820
31  [3, 1, 2, 7, 4, 8, 5, 9, 10, 6]   9903
87  [10, 8, 2, 7, 3, 1, 9, 4, 6, 5]  10092
Total distance travelled to cover the final route of 
 Bangalore => Hyderabad => Kolkata => Agra => Varanasi => Delhi => Jaipur => Mumbai => Udaipur => Mysore => Chennai => Bangalore is 9613
starting crossover
starting Mutation
Completed 3 runs of GA.
below are the best chromoso

starting Mutation
Completed 20 runs of GA.
below are the best chromosomes:
                              Route  Cost
49  [9, 3, 1, 4, 7, 8, 2, 10, 5, 6]  7104
69  [10, 9, 1, 4, 3, 7, 2, 8, 5, 6]  7605
68  [10, 9, 1, 4, 3, 7, 2, 8, 5, 6]  7605
Total distance travelled to cover the final route of 
 Bangalore => Mumbai => Udaipur => Jaipur => Delhi => Agra => Kolkata => Varanasi => Hyderabad => Chennai => Mysore => Bangalore is 7104


In [32]:

final_route = []
for i in best_solution:
    final_route.append(cities_mapping[i])
final_route = [starting_point] + final_route + [starting_point]
print("Total distance travelled to cover the final route of \n {} is {}".format(" => ".join(final_route),best_cost))


Total distance travelled to cover the final route of 
 Bangalore => Mumbai => Udaipur => Jaipur => Delhi => Agra => Kolkata => Varanasi => Hyderabad => Chennai => Mysore => Bangalore is 7104
